# XBeach bulk data forcing

Demo notebook with usage examples

## Overview

XBeach is normally run in the nearshore, over relative small spatial scales. On these
scales, forcing input data to the model can often be assumed to be "stationary". For
example, in a typical domain of *O(1) km*, input forcing such as wind, tide elevation
and wave boundary could be considered to be relatively constant across the domain.

Rompy-XBeach provides Data objects to support bulk forcing, i.e., forcing data defined
for a single point location by tabular data. In contrast to other data objects such as
the grid- or station-based ones, point-based objects deal with data sources that do
not require spatial coordinates and can be represented for example by a CSV file or a
Pandas DataFrame. Source objects are available to represent these data types.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from datetime import datetime, timedelta
import pandas as pd
import xarray as xr
import shutil

import warnings
warnings.filterwarnings("ignore")

In [3]:
def generate_output_directory(path="bulk-forcing-demo"):
    """Generate the output dir for the examples, removing it if it already exists."""
    outdir = Path(path)
    if outdir.exists():
        shutil.rmtree(outdir)
    outdir.mkdir()
    return outdir

In [4]:
datadir = Path("../../../rompy-xbeach/tests/data")

Let's start by defining common grid and timerange objects to use with the different examples in this Notebook.

In [5]:
from rompy.core.time import TimeRange
from rompy_xbeach.grid import RegularGrid

In [6]:
# Model times
times = TimeRange(start="2023-01-01T00", end="2023-01-01T12", interval="1h")

# Model grid
grid = RegularGrid(
    ori=dict(x=115.594239, y=-32.641104, crs="epsg:4326"),
    alfa=347.0,
    dx=10,
    dy=15,
    nx=230,
    ny=220,
    crs="28350",
)

## Wind forcing

Bulk wind forcing can be prescribed by the **WindPoint** object which currently supports
input timeseries data from CSV file or a Pandas DataFrame object.

In [7]:
from rompy.core.source import SourceTimeseriesCSV
from rompy_xbeach.forcing import WindPoint, WindVector, WindScalar

In [8]:
# Generate the output directory
destdir = generate_output_directory()

# Input data source definition
source = SourceTimeseriesCSV(
    filename=datadir / "wind.csv",
    tcol="time",
)

# Wind object
wind = WindPoint(
    source=source,
    wind_vars=WindScalar(spd="wspd", dir="wdir"),
)

# Generating the forcing data
namelist = wind.get(destdir=destdir, grid=grid, time=times)
namelist

{'windfile': 'wind-20230101T000000-20230101T120000.txt'}

In [9]:
# Check the content of the generated wind file
windfile = destdir / namelist["windfile"]
print(windfile.read_text())

      0.00       7.24     149.87
   3600.00       7.15     150.90
   7200.00       7.11     154.32
  10800.00       7.27     159.03
  14400.00       7.54     163.52
  18000.00       7.97     167.57
  21600.00       8.36     170.24
  25200.00       8.77     172.81
  28800.00       9.17     174.48
  32400.00       9.54     174.92
  36000.00       9.78     177.59
  39600.00      10.09     175.46
  43200.00      10.19     172.35



## Tide forcing

### Constituents

Tide timeseries can be generated with the **TideConsPoint** object from tabular constituents
data such as harmonic constituents for a site. These data can currently be prescribed
from CSV files with columns representing tide elevation amplitude and phase for the
different constituents. Similar to the TideGrid object, oceantide is used to convert
these into elevation timeseries.

In [10]:
from rompy_xbeach.source import SourceTideConsPointCSV
from rompy_xbeach.forcing import TideConsPoint

In [11]:
# Generate the output directory
destdir = generate_output_directory()

# Input data source definition
source = SourceTideConsPointCSV(
    filename=datadir / "tide_cons_station.csv",
    acol="amplitude",
    pcol="phase",
    ccol="constituent",
)

# Tide object
tide = TideConsPoint(source=source)

# Generating the forcing data
namelist = tide.get(destdir=destdir, grid=grid, time=times)
print(namelist)

{'zs0file': 'tide-20230101T000000-20230101T120000.txt', 'tideloc': 1, 'tidelen': 13}


In [12]:
# Check the content of the generated tide elevation file
zs0file = destdir / namelist["zs0file"]
print(zs0file.read_text())

      0.00      -0.20
   3600.00      -0.15
   7200.00      -0.09
  10800.00      -0.02
  14400.00       0.06
  18000.00       0.13
  21600.00       0.19
  25200.00       0.24
  28800.00       0.26
  32400.00       0.26
  36000.00       0.23
  39600.00       0.20
  43200.00       0.15



### Water level

Alternatively, tide input (or, more generically, water level boundary) can be prescribed
from timeseris type data instead of tide constituents. Three objects are available to
support gridded-, station- and point-based water level timeseries data. Here we are
going to demonstrate **WaterLevelPoint** which supports point-based ("bulk") type data.

In [13]:
from rompy_xbeach.forcing import WaterLevelPoint

In [14]:
# Generate the output directory
destdir = generate_output_directory()

# Input data source definition
source = SourceTimeseriesCSV(filename=datadir / "ssh.csv", tcol="time")

# Tide object
tide = WaterLevelPoint(source=source, variables=["ssh"])

# Generating the forcing data
namelist = tide.get(destdir=destdir, grid=grid, time=times)
print(namelist)

{'zs0file': 'tide-20230101T000000-20230101T120000.txt', 'tideloc': 1, 'tidelen': 25}


In [15]:
# Check the content of the generated tide elevation file
zs0file = destdir / namelist["zs0file"]
print(zs0file.read_text())

      0.00      -0.61
   3600.00      -0.48
   7200.00      -0.33
  10800.00      -0.16
  14400.00       0.03
  18000.00       0.25
  21600.00       0.47
  25200.00       0.68
  28800.00       0.87
  32400.00       1.01
  36000.00       1.08
  39600.00       1.06
  43200.00       0.95
  46800.00       0.75
  50400.00       0.47
  54000.00       0.14
  57600.00      -0.20
  61200.00      -0.52
  64800.00      -0.79
  68400.00      -0.99
  72000.00      -1.11
  75600.00      -1.14
  79200.00      -1.10
  82800.00      -1.00
  86400.00      -0.85



## Wave boundary

Wave boundary can be defined from timeseries of wave parameters defined form either CSV
or pandas DataFrame input source data. Two types of XBeach boundary are currently supported
with timeseries type source:

- *JONS*: **BoundaryPointParamJons**
- *JONSTABLE*: **BoundaryPointParamJonstable**

### JONS

In [16]:
from rompy_xbeach.boundary import BoundaryPointParamJons

In [17]:
# Generate the output directory
destdir = generate_output_directory()

# Input data source definition
source = SourceTimeseriesCSV(filename=datadir / "wave-params-20230101.csv")

# Wind object
wb = BoundaryPointParamJons(
    source=source,
    hm0="phs1",
    tp="ptp1",
    mainang="pdp1",
    gammajsp="ppe1",
    dspr="pspr1",
)

# Generating the forcing data
namelist = wb.get(destdir=destdir, grid=grid, time=times)
namelist

{'wbctype': 'jons', 'bcfile': 'jons-filelist.txt'}

In [18]:
# Check the content of the generated boundary file
bcfile = destdir / namelist["bcfile"]
filelist = bcfile.read_text()
print(filelist)

FILELIST
3600 1 jons-20230101T000000.txt
3600 1 jons-20230101T010000.txt
3600 1 jons-20230101T020000.txt
3600 1 jons-20230101T030000.txt
3600 1 jons-20230101T040000.txt
3600 1 jons-20230101T050000.txt
3600 1 jons-20230101T060000.txt
3600 1 jons-20230101T070000.txt
3600 1 jons-20230101T080000.txt
3600 1 jons-20230101T090000.txt
3600 1 jons-20230101T100000.txt
3600 1 jons-20230101T110000.txt



In [19]:
# Check the content of one of the filelist files
jonfile = destdir / filelist.split("\n")[1].split()[-1]
print(jonfile.read_text())

Hm0 = 1.01011
mainang = 244.342
gammajsp = 2.52055
Tp = 12.2993
s = 33.9251



### JONSTABLE

In [20]:
from rompy_xbeach.boundary import BoundaryPointParamJonstable

In [21]:
# Generate the output directory
destdir = generate_output_directory()

# Input data source definition
source = SourceTimeseriesCSV(filename=datadir / "wave-params-20230101.csv")

# Wind object
wb = BoundaryPointParamJonstable(
    source=source,
    hm0="phs1",
    tp="ptp1",
    mainang="pdp1",
    gammajsp="ppe1",
    dspr="pspr1",
)

# Generating the forcing data
namelist = wb.get(destdir=destdir, grid=grid, time=times)
namelist

{'wbctype': 'jonstable',
 'bcfile': 'jonstable-20230101T000000-20230101T120000.txt'}

In [22]:
# Check the content of the generated boundary file
bcfile = destdir / namelist["bcfile"]
print(bcfile.read_text())

1.01011 12.2993 244.342 2.52055 33.9251 3600 1
0.968176 13.7877 247.584 3.02407 41.2201 3600 1
0.994085 14.1542 248.386 3.02558 48.7203 3600 1
1.07939 13.5109 246.407 2.65635 46.7938 3600 1
1.13343 15.3336 250.89 2.79238 54.9961 3600 1
1.36986 15.4369 251.308 2.40611 50.0111 3600 1
1.46595 15.2322 251.179 2.30742 48.5546 3600 1
1.55301 15.0677 251.001 2.18789 47.001 3600 1
1.60305 14.9164 250.805 2.12522 46.4984 3600 1
1.63845 14.77 250.583 2.06828 46.1284 3600 1
1.66568 14.625 250.334 2.00733 45.8816 3600 1
1.68705 14.4773 250.034 1.94841 45.9753 3600 1
1.70383 14.321 249.769 1.89275 45.5342 3600 1



## Define a model Config

Use the bulk forcing instances created above to define a Rompy XBeach config object

In [23]:
from rompy_xbeach.data import XBeachBathy
from rompy_xbeach.interpolate import RegularGridInterpolator
from rompy_xbeach.data import SeawardExtensionLinear
from rompy_xbeach.source import SourceGeotiff

### Define the bathy

In [24]:
# Interpolator
interpolator = RegularGridInterpolator(
    kwargs=dict(
        method="linear",
        fill_value=None,
    ),
)

# Seaward extension
extension = SeawardExtensionLinear(
    depth=25,
    slope=0.1,
)

# Bathy object
bathy = XBeachBathy(
    source=SourceGeotiff(filename=datadir / "bathy.tif"),
    posdwn=False,
    interpolator=interpolator,
    extension=extension,
    left=5,
    right=5,
)

### Instantiate the Config

In [25]:
from rompy_xbeach.config import Config

In [26]:
config = Config(
    grid=grid,
    bathy=bathy,
    input=dict(wave=wb, wind=wind, tide=tide),
    front="abs_2d",
    back="abs_2d",
    left="neumann",
    right="neumann",
    rugdepth=0.011,
    scheme="warmbeam",
    order=1,
    lateralwave="wavecrest",
    random=True,
    zs0=0.0,
    hmin=0.01,
    wci=False,
    alpha=1,
    delta=0.0,
    n=10,
    rho=1025,
    g=9.81,
    thetamin=-80,
    thetamax=80,
    dtheta=10.0,
    beta=0.1,
    roller=True,
    gamma=0.55,
    gammax=1.0,
    sedtrans=False,
    morfac=1.0,
    morphology=False,
    cf=0.01,
    paulrevere="land",
    eps=0.01,
    epsi=0.001,
    cfl=0.8,
    umin=0.1,
    oldhu=True,
    tstart=0,
    tintm=3600.0,
    outputformat="netcdf",
    ncfilename="xboutput_test.nc",
)

### Generate the workspace

In [27]:
from rompy.model import ModelRun

In [28]:
# Generate the output directory
destdir = generate_output_directory()

modelrun = ModelRun(
    run_id="test1",
    period=times,
    output_dir=destdir,
    config=config,
)

rundir = modelrun()

INFO:rompy.model:
INFO:rompy.model:-----------------------------------------------------
INFO:rompy.model:Model settings:
INFO:rompy.model:
run_id: test1
period: 
	Start: 2023-01-01 00:00:00
	End: 2023-01-01 12:00:00
	Duration: 12:00:00
	Interval: 1:00:00
	Include End: True

output_dir: bulk-forcing-demo
config: <class 'rompy_xbeach.config.Config'>

INFO:rompy.model:-----------------------------------------------------
INFO:rompy.model:Generating model input files in bulk-forcing-demo
INFO:rompy_xbeach.config:Generating wave boundary data
INFO:rompy_xbeach.config:Generating wind forcing data
INFO:rompy_xbeach.config:Generating tide forcing data
INFO:rompy.model:
INFO:rompy.model:Successfully generated project in /source/csiro/rompy-notebooks/notebooks/xbeach/bulk-forcing-demo/test1
INFO:rompy.model:-----------------------------------------------------


### Check the workspace files

In [29]:
modeldir = Path(modelrun.output_dir) / modelrun.run_id

sorted(modeldir.glob("*"))

[PosixPath('bulk-forcing-demo/test1/bathy.txt'),
 PosixPath('bulk-forcing-demo/test1/jonstable-20230101T000000-20230101T120000.txt'),
 PosixPath('bulk-forcing-demo/test1/params.txt'),
 PosixPath('bulk-forcing-demo/test1/tide-20230101T000000-20230101T120000.txt'),
 PosixPath('bulk-forcing-demo/test1/wind-20230101T000000-20230101T120000.txt'),
 PosixPath('bulk-forcing-demo/test1/xdata.txt'),
 PosixPath('bulk-forcing-demo/test1/ydata.txt')]

### Check the params file

In [30]:
params = modeldir / "params.txt"

print(params.read_text())

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%% XBeach parameter settings input file
%%%
%%% Date: 2025-03-12 04:50:40.577933
%%% User: rafael-XPS
%%% Template: /source/csiro/rompy-xbeach/src/rompy_xbeach/templates/base
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

zs0 = 0.0
front = abs_2d
back = abs_2d
left = neumann
right = neumann
lateralwave = wavecrest
rugdepth = 0.011
scheme = warmbeam
order = 1
random = 1
hmin = 0.01
wci = 0
alpha = 1.0
delta = 0.0
n = 10.0
rho = 1025.0
g = 9.81
thetamin = -80.0
thetamax = 80.0
dtheta = 10.0
beta = 0.1
roller = 1
gamma = 0.55
gammax = 1.0
sedtrans = 0
morfac = 1.0
morphology = 0
cf = 0.01
eps = 0.01
epsi = 0.001
cfl = 0.8
umin = 0.1
oldhu = 1
outputformat = netcdf
ncfilename = xboutput_test.nc
tstart = 0.0
tintm = 3600.0
paulrevere = land
tstop = 43200.0
tunits = seconds since 2023-01-01 00:00:00
wbctype = jonstable
bcfile = jonstable-20230101T000000-20230101T120000.txt
windfile = wi